In [1]:
#####Modelling Reciprocating Relationships with Hawkes Processes#####
import numpy as np
import pandas as pd
import matplotlib.pyplot  as plt
import numpy.matlib
import scipy.linalg
import itertools
import seaborn as sns
from scipy import sparse
from scipy.stats import norm
from pandas.tools.plotting import scatter_matrix
from numpy.random import *
from scipy import optimize

#np.random.seed(98537)

In [2]:
##多項分布の乱数を生成する関数
def rmnom(pr, n, k, no, pattern):
    z_id = np.argmax((np.cumsum(pr, axis=1) >= np.random.uniform(0, 1, n)[:, np.newaxis]), axis=1)
    if pattern==1:
        Z = sparse.coo_matrix((np.repeat(1, n), (no, np.array(z_id))), shape=(n, k))   #スパース行列の設定
        return z_id, Z
    return z_id

In [3]:
####データの発生####
##データの設定
k1 = 9
k2 = 7
hh = 5000
item = 3000
pt = np.random.poisson(np.random.gamma(20.0, 1/0.25, hh), hh); pt[pt <= 5] = 5
hhpt = np.sum(pt)

In [4]:
##IDとインデックスを設定
#IDを設定
d_id = np.repeat(range(hh), pt)
pt_id = np.array(list(itertools.chain(*[np.array(range(pt[i]), dtype="int") for i in range(hh)])))

#インデックスを設定
d_list = [i for i in range(hh)]
d_vec = [i for i in range(hh)]
pt_list = [j for j in range(np.max(pt))]
pt_n = np.repeat(0, np.max(pt))
for i in range(hh):
    d_list[i] = np.array(np.where(d_id==i)[0], dtype="int")
    d_vec[i] = np.repeat(1, pt[i])
for j in range(np.max(pt)):
    pt_list[j] = np.array(np.where(pt_id==j)[0], dtype="int")
    pt_n[j] = pt_list[j].shape[0]
max_index = np.array([np.max(d_list[i]) for i in range(hh)])

In [5]:
##アイテムの割当を生成
#セグメント割当を生成
topic = 25
phi = np.random.dirichlet(np.repeat(0.5, item), topic)
theta = np.random.dirichlet(np.repeat(2.5, topic), hh)
z = np.dot(np.array([np.random.multinomial(1, theta[i, :], 1) for i in range(hh)]).reshape(hh, topic), range(topic))

#多項分布からアイテムを生成
item_id = np.zeros(hhpt, dtype='int')
for i in range(hh):
    if i%1000==0:
        print(i)
    item_id[d_list[i]] = np.dot(np.random.multinomial(1, phi[z[i], :], pt[i]), range(item))
    
#インデックスの設定
item_list = [j for j in range(item)]
item_vec = [j for j in range(item)]
item_n = np.repeat(0, item)
for j in range(item):
    item_list[j] = np.array(np.where(item_id==j)[0], dtype="int")
    item_vec[j] = np.repeat(1, len(item_list[j]))
    item_n[j] = len(item_list[j])

0
1000
2000
3000
4000


In [6]:
##応答変数が妥当な値になるまで繰り返す
rp = 0
while True:
    rp = rp + 1

    ##パラメータを生成
    #潜在変数を生成
    theta1 = np.random.dirichlet(np.repeat(2.0, k1), 1).reshape(-1)
    theta2 = np.random.dirichlet(np.repeat(2.0, k2), 1).reshape(-1)
    Z1 = np.random.multinomial(1, theta1, hh)
    Z2 = np.random.multinomial(1, theta2, item)
    z1_vec = np.dot(Z1, np.arange(k1)); z2_vec = np.dot(Z2, np.arange(k2))
    thetat1 = theta1.copy(); thetat2 = theta2.copy()
    
    #ガンマ分布の事前分布のパラメータ
    alpha1 = 0.75; beta1 = 0.15
    alpha2 = 0.75; beta2 = 0.125
    alpha3 = 0.5; beta3 = 0.125

    #モデルパラメータを生成
    gamma1 = np.random.gamma(alpha1, 1/beta1, k1)
    gamma2 = np.random.gamma(alpha2, 1/beta2, k2)
    gamma3 = np.random.gamma(alpha3, 1/beta3, k1*k2).reshape(k1, k2)
    gammat1 = gamma1.copy(); gammat2 = gamma2.copy(); gammat3 = gamma3.copy()

    ##応答変数を生成
    #モデルの期待値
    mu1 = gamma1[z1_vec[d_id], ]
    mu2 = gamma2[z2_vec[item_id], ]
    mu3 = np.sum(gamma3[z1_vec[d_id], ] * Z2[item_id, ], axis=1)
    mu = mu1 + mu2 + mu3
    mut = mu.copy()
    
    #指数分布から応答変数を生成
    y = np.random.exponential(mu, hhpt)

    #break条件
    print([rp, np.max(y), np.min(y)])
    if (np.max(y) < 200):
        break

[1, 483.3719666806564, 2.4946956624520862e-06]
[2, 365.6358899986922, 2.4782119890855135e-05]
[3, 701.5725995722174, 2.2460749586929892e-05]
[4, 612.2346410514483, 1.7053752767291935e-05]
[5, 575.7541763398, 7.026723324605891e-06]
[6, 329.5613071693653, 0.00011488753248240235]
[7, 232.98730091387398, 1.882447816534237e-05]
[8, 605.3751556022385, 7.063722617080642e-07]
[9, 436.99009290081034, 5.049622288974226e-05]
[10, 349.59700613231985, 3.9173132079238645e-05]
[11, 359.582887060304, 1.1778944642192486e-06]
[12, 480.1054107312187, 8.964750622393998e-05]
[13, 367.7997475639235, 9.768562766266124e-06]
[14, 435.4860428849164, 4.749840439682301e-05]
[15, 633.1017388855046, 3.977853319890485e-05]
[16, 452.62136508374573, 3.445441639025675e-05]
[17, 342.46909954535346, 5.767351183236743e-05]
[18, 398.9307923349864, 4.476594086767346e-06]
[19, 450.24624628993297, 8.906358156116224e-06]
[20, 528.2406234877602, 2.65609937869042e-05]
[21, 452.53308597579905, 1.6527279477757444e-05]
[22, 382.622

[175, 321.76682835212404, 6.964903898882203e-06]
[176, 412.04219629760524, 3.3388598775268498e-06]
[177, 382.1121442278581, 1.3413199012052851e-05]
[178, 301.22458478030705, 2.1861930277400397e-06]
[179, 490.355394984814, 3.696732153857705e-05]
[180, 368.32067530562034, 7.204580544942162e-06]
[181, 264.22342629898776, 6.123142382678677e-06]
[182, 384.15498831172664, 3.424762901410751e-05]
[183, 402.27068820340503, 6.530083774159166e-06]
[184, 407.8821287548772, 1.0466247345838626e-05]
[185, 360.3776779529445, 2.4140977620309182e-05]
[186, 417.35068819709755, 7.779215689881226e-05]
[187, 439.74178190089157, 3.556692536868846e-05]
[188, 336.2000397458749, 4.15066786487287e-05]
[189, 252.96245410137647, 1.4755509842774937e-05]
[190, 496.4932514833786, 2.712414068266661e-06]
[191, 373.2443559655838, 4.9265160002055466e-05]
[192, 380.584459283319, 1.0771436109187965e-05]
[193, 479.91188937800143, 6.402344193355151e-06]
[194, 327.6360154856856, 8.310620715591735e-05]
[195, 284.3710737133985,

In [7]:
####マルコフ連鎖モンテカルロ法でパラメータを推定####
##切断指数分布の乱数を生成する関数
def rtexp(gamma, a, b):
    #切断指数分布の乱数を生成
    FA = scipy.stats.expon.cdf(a, scale=gamma)
    FB = scipy.stats.expon.cdf(b, scale=gamma)
    par = scipy.stats.expon.ppf(np.random.uniform(0, 1, a.shape[0])*(FB-FA)+FA, scale=gamma)
    return par

In [8]:
##データの設定
R = 2000
keep = 4
burnin = int(500/keep)
iter = 0
disp = 10
e1 = 0.001
e2 = 0.0025
L = 3

In [9]:
##事前分布の設定
alpha01 = 1.0; beta01 = 1.0
alpha02 = 1.0; beta02 = 1.0
alpha03 = 1.0; beta03 = 1.0
alpha11 = 1.0
alpha12 = 1.0

In [10]:
##パラメータの真値
#潜在変数の真値
theta1 = thetat1.copy()
theta2 = thetat2.copy()
Zi1 = Z1.copy()
Zi2 = Z2.copy()
z1_vec = np.dot(Zi1, np.arange(k1))
z2_vec = np.dot(Zi2, np.arange(k2))

#モデルパラメータの真値
theta1 = thetat1.copy()
theta2 = thetat2.copy()
gamma1 = gammat1.copy()
gamma2 = gammat2.copy()
gamma3 = gammat3.copy()

#モデルの期待値
mu1 = gamma1[z1_vec[d_id]]
mu2 = gamma2[z2_vec[item_id]]
mu3 = np.sum(gamma3[z1_vec[d_id], ] * Z2[item_id, ], axis=1)

In [11]:
##パラメータの初期値
#潜在変数の初期値
theta1 = np.random.dirichlet(np.repeat(5.0, k1), 1).reshape(-1)
theta2 = np.random.dirichlet(np.repeat(5.0, k2), 1).reshape(-1)
Zi1 = np.random.multinomial(1, theta1, hh)
Zi2 = np.random.multinomial(1, theta2, item)
z1_vec = np.dot(Zi1, np.arange(k1))
z2_vec = np.dot(Zi2, np.arange(k2))

#モデルパラメータの初期値
a = np.mean(y) / 3.0; b = 1.0
gamma1 = np.random.gamma(a, 1/b, k1)
gamma2 = np.random.gamma(a, 1/b, k2)
gamma3 = np.random.gamma(a, 1/b, k1*k2).reshape(k1, k2)

In [12]:
##パラメータの格納用配列
#潜在変数の格納用配列
SEG1 = np.zeros((hh, k1))
SEG2 = np.zeros((item, k2))

#モデルパラメータの格納用配列
GAMMA1 = np.zeros((int(R/keep), k1))
GAMMA2 = np.zeros((int(R/keep), k2))
GAMMA3 = np.zeros((k1, k2, int(R/keep)))

In [13]:
##対数尤度の基準値
#1パラメータモデルの対数尤度
LLst = np.sum(scipy.stats.expon.logpdf(x=y, scale=np.mean(y)))
print(LLst)

#真値での対数尤度
LLbest = np.sum(scipy.stats.expon.logpdf(x=y, scale=mut))
print(LLbest)

-1268029.5183967808
-1203329.3793903063


In [14]:
####ギブスサンプリングでパラメータをサンプリング####
for rp in range(R):
    ##ユーザーの潜在変数をサンプリング
    #潜在変数の割当確率
    LLho = np.zeros((hhpt, k1)); Lho = np.zeros((hh, k1))
    Lambda = np.full((hhpt, k1), gamma1) + mu2[:, np.newaxis] + gamma3.T[z2_vec[item_id], ]
    for j in range(k1):
        LLho[:, j] = scipy.stats.expon.logpdf(x=y, scale=Lambda[:, j])
    for i in range(hh):
        Lho[i, ] = np.sum(LLho[d_list[i], ], axis=0)
    Lho = theta1 * np.exp(Lho - np.max(Lho, axis=1)[:, np.newaxis])
    Prob = Lho / np.sum(Lho, axis=1)[:, np.newaxis]

    #多項分布から潜在変数をサンプリング
    res = rmnom(Prob, hh, k1, np.arange(hh), 1)
    z1_vec = np.array(res[0], dtype="int")
    Zi1 = np.array(res[1].todense(), dtype="int")
    z1 = z1_vec[d_id]
    mu1 = gamma1[z1_vec[d_id]]

    ##アイテムの潜在変数をサンプリング
    #潜在変数の割当確率
    LLho = np.zeros((hhpt, k2)); Lho = np.zeros((item, k2))
    Lambda = mu1[:, np.newaxis] + np.full((hhpt, k2), gamma2) + gamma3[z1_vec[d_id], ]
    for j in range(k2):
        LLho[:, j] = scipy.stats.expon.logpdf(x=y, scale=Lambda[:, j])   
    for i in range(item):
        Lho[i, ] = np.sum(LLho[item_list[i], ], axis=0)
    Lho = theta2 * np.exp(Lho - np.max(Lho, axis=1)[:, np.newaxis])
    Prob = Lho / np.sum(Lho, axis=1)[:, np.newaxis]

    #多項分布から潜在変数をサンプリング
    res = rmnom(Prob, item, k2, np.arange(item), 1)
    z2_vec = np.array(res[0], dtype="int")
    Zi2 = np.array(res[1].todense(), dtype="int")
    z2 = z2_vec[item_id]
    mu2 = gamma2[z2_vec[item_id]]


    ##ガンマ分布よりユーザーパラメータをサンプリング
    #補助変数lambdaを更新
    mu = mu1 + mu2 + mu3
    Lambda = mu1 / mu

    #事後分布のパラメータを更新
    lambda_y = Lambda * y
    s1 = np.repeat(0.0, k1); v1 = np.repeat(0.0, k1)
    for j in range(k1):
        index = np.where(z1==j)[0]
        s1[j] = np.sum(lambda_y[index]) + alpha01
        v1[j] = np.sum(index.shape[0]) + beta01

    #パラメータをサンプリング
    gamma1 = np.random.gamma(s1, 1/v1, k1)
    mu1 = gamma1[z1_vec[d_id]]

    ##ガンマ分布よりアイテムパラメータをサンプリング
    #補助変数lambdaを更新
    mu = mu1 + mu2 + mu3
    Lambda = mu2 / mu

    #事後分布のパラメータを更新
    lambda_y = Lambda * y
    s2 = np.repeat(0.0, k2); v2 = np.repeat(0.0, k2)
    for j in range(k2):
        index = np.where(z2==j)[0]
        s2[j] = np.sum(lambda_y[index]) + alpha02
        v2[j] = np.sum(index.shape[0]) + beta02

    #パラメータをサンプリング
    gamma2 = np.random.gamma(s2, 1/v2, k2)
    mu2 = gamma2[z2_vec[item_id]]

    ##ガンマ分布よりユーザー-アイテムパラメータをサンプリング
    #補助変数lambdaを更新
    mu = mu1 + mu2 + mu3
    Lambda = (mu3 / mu)

    #事後分布のパラメータを更新
    temp_z1 = Zi1[d_id, ]; temp_z2 = Zi2[item_id, ]
    lambda_y = (Lambda * y)[:, np.newaxis]
    for j in range(k2):
        temp_z = temp_z1 * temp_z2[:, j][:, np.newaxis]
        s3 = np.sum(lambda_y * temp_z, axis=0) + alpha03
        v3 = np.sum(temp_z, axis=0) + beta03

        #パラメータをサンプリング
        gamma3[:, j] = np.random.gamma(s3, 1/v3, k1)
    mu3 = np.sum(gamma3[z1_vec[d_id], ] * Z2[item_id, ], axis=1)

    ##ディリクレ分布から混合率を更新
    theta1 = np.random.dirichlet(np.sum(Zi1, axis=0) + alpha11, 1).reshape(-1)
    theta2 = np.random.dirichlet(np.sum(Zi2, axis=0) + alpha12, 1).reshape(-1)


    ##パラメータの格納とサンプリング結果の表示
    #サンプリング結果の格納
    if rp%keep==0:
        mkeep = rp//keep
        GAMMA1[mkeep, ] = gamma1
        GAMMA2[mkeep, ] = gamma2
        GAMMA3[:, :, mkeep] = gamma3
        if rp >= burnin:
            SEG1 = SEG1 + Zi1
            SEG2 = SEG2 + Zi2

    if rp%disp==0:
        #対数尤度を更新
        mu = mu1 + mu2 + mu3
        LL = np.sum(scipy.stats.expon.logpdf(x=y, scale=mu))

        #サンプリング結果の表示
        print(rp)
        print(np.round(np.array([LL, LLst, LLbest]), 1))

0
[-1238329.6 -1268029.5 -1203329.4]
10
[-1235396.7 -1268029.5 -1203329.4]
20
[-1234752.8 -1268029.5 -1203329.4]
30
[-1241647.8 -1268029.5 -1203329.4]
40
[-1237517.5 -1268029.5 -1203329.4]
50
[-1232396.7 -1268029.5 -1203329.4]
60
[-1230373.2 -1268029.5 -1203329.4]
70
[-1230705.1 -1268029.5 -1203329.4]
80
[-1231251.2 -1268029.5 -1203329.4]
90
[-1230686.4 -1268029.5 -1203329.4]
100
[-1243995.9 -1268029.5 -1203329.4]
110
[-1252490.  -1268029.5 -1203329.4]
120
[-1238444.  -1268029.5 -1203329.4]
130
[-1239477.8 -1268029.5 -1203329.4]
140
[-1240791.8 -1268029.5 -1203329.4]
150
[-1234019.4 -1268029.5 -1203329.4]
160
[-1233448.2 -1268029.5 -1203329.4]
170
[-1240133.9 -1268029.5 -1203329.4]
180
[-1241026.5 -1268029.5 -1203329.4]
190
[-1242717.1 -1268029.5 -1203329.4]
200
[-1242199.4 -1268029.5 -1203329.4]
210
[-1241008.7 -1268029.5 -1203329.4]
220
[-1238808.  -1268029.5 -1203329.4]
230
[-1244108.5 -1268029.5 -1203329.4]
240
[-1243957.6 -1268029.5 -1203329.4]
250
[-1241114.4 -1268029.5 -1203329.